<a href="https://colab.research.google.com/github/pevu97/Jobb-Offer-Classifier/blob/main/notebook/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import requests
!pip install beautifulsoup4

def get_job_details(job_id):
  job_id = job_id
  url = f"https://nofluffjobs.com/api/posting/{job_id}"

  headers = {
      "User-Agent": "Mozilla/5.0",
      "Accept": "application/json"
  }

  response = requests.get(url, headers=headers)

  return response.json()


def get_job_list(sentence):
  url = "https://nofluffjobs.com/api/search/posting"
  params = {
      "pageFrom": 1,
      "pageTo": 1,
      "pageSize": 20,
      "salaryCurrency": "PLN",
      "salaryPeriod": "month",
      "region": "pl",
      "language": "pl-PL",
      "withSalaryMatch": "true"
  }

  headers = {
      "Content-Type": "application/infiniteSearch+json",
      "User-Agent": "Mozilla/5.0"
  }

  payload = {
      "rawSearch": f"requirement={sentence}"
  }

  response = requests.post(url, headers=headers, params=params, json=payload)

  return response.json()


In [7]:
offer_list = get_job_list("python")
type(offer_list)

dict

In [32]:
offer_list['postings'][215]['id']

'senior-fullstack-developer-python-javascript-order-group-west-pomeranian'

In [16]:
jobs_titles = []
for offer in offer_list['postings']:
  jobs_titles.append(offer['id'])

In [19]:
job_details = get_job_details(jobs_titles[0])

In [48]:
keywords = [
    "python", "java", "javascript", "react", "django", "sql", "node", "devops", "qa",
    "junior", "mid", "entry", "trainee", "intern"
]


title_list = []
for key in keywords:
  offer_list = get_job_list(key)['postings']
  for offer in offer_list:
    title_list.append(offer['id'])
len(title_list)

1929

In [49]:
title_list[1]

'senior-python-full-stack-developer-link-group-lower-silesian'

In [50]:
job_details = []

for title in title_list:
  job_details.append(get_job_details(title))


In [46]:
len(job_details)

1974

In [51]:
from collections import Counter

labels = [offer['basics']['seniority'][0].lower() for offer in job_details if offer.get('basics', {}).get('seniority')]
print(Counter(labels))

Counter({'senior': 917, 'mid': 636, 'junior': 178, 'trainee': 144, 'expert': 54})


In [68]:
jobs = {'senior':[], 'mid':[], 'junior':[]}

In [69]:
for offer in job_details:
  if offer.get('basics', {}).get('seniority'):
    if 'senior' in offer['basics']['seniority'][0].lower():
      jobs['senior'].append(offer)
    if 'mid' in offer['basics']['seniority'][0].lower():
      jobs['mid'].append(offer)
    if 'junior' in offer['basics']['seniority'][0].lower():
      jobs['junior'].append(offer)
    if 'trainee' in offer['basics']['seniority'][0].lower():
      jobs['junior'].append(offer)
    if 'expert' in offer['basics']['seniority'][0].lower():
      jobs['senior'].append(offer)

In [71]:
len(jobs['senior'])

971

In [102]:
jobs['senior'][0]['title']

'Senior Python Full-Stack Developer'

In [110]:

from bs4 import BeautifulSoup


text = jobs['senior'][0]['requirements']['description']

soup = BeautifulSoup(text, 'html.praser')
clean_text = soup.get_text()
print(clean_text)

FeatureNotFound: Couldn't find a tree builder with the features you requested: html.praser. Do you need to install a parser library?

In [96]:
jobs['senior'][0]['requirements']['musts'][0]

{'value': 'Python', 'type': 'main'}

In [105]:
for key in jobs['senior'][0]['requirements']['musts'][0]:
  print(jobs['senior'][0]['requirements']['musts'][0][key])

Python
main
